# TODO: Zklend

In [ ]:
from typing import Dict
import collections
import decimal

import IPython.display
import pandas

import constants
import db

## Load and prepare events

In [ ]:
# Establish the connection.
connection = db.establish_connection()

In [ ]:
# Load all Zklend events.
zklend_events = pandas.read_sql(
    sql =  """
        SELECT
            *
        FROM
            starkscan_events
        WHERE
            from_address='{}';
    """.format(constants.Protocol.ZKLEND.value),
    con = connection,
)

In [ ]:
# Close the connection.
connection.close()

In [ ]:
# TODO: move to constants?
RELEVANT_EVENTS = {
    'Deposit',
    'Withdrawal',
    'CollateralEnabled',
    'CollateralDisabled',
    'Borrowing',
    'Repayment',
    'Liquidation',
    'AccumulatorsSync',
}

# Remove redundant data, sort remaining data and set index.
zklend_events.drop(
    zklend_events[~zklend_events['key_name'].isin(RELEVANT_EVENTS)].index,
    inplace = True,
)
zklend_events.sort_values('block_number', inplace = True)
zklend_events.set_index('id', inplace = True)

# Process events

Events examples:
- [Deposit](https://starkscan.co/event/0x036185142bb51e2c1f5bfdb1e6cef81f8ea87fd4d777990014249bf5435fd31b_3)
- [AccumulatorsSync](https://starkscan.co/event/0x029628b89875a98c1c64ae206e7eb65669cb478a24449f3485f5e98aba6204dc_0)
- [CollateralEnabled](https://starkscan.co/event/0x036185142bb51e2c1f5bfdb1e6cef81f8ea87fd4d777990014249bf5435fd31b_6)
- [CollateralDisabled](https://starkscan.co/event/0x0049b445bed84e0118795dbd22d76610ccac2ad626f8f04a1fc7e38113c2afe7_0)
- [Withdrawal](https://starkscan.co/event/0x03472cf7511687a55bc7247f8765c4bbd2c18b70e09b2a10a77c61f567bfd2cb_4)
- [Borrowing](https://starkscan.co/event/0x076b1615750528635cf0b63ca80986b185acbd20fa37f0f2b5368a4f743931f8_3)
- [Repayment](https://starkscan.co/event/0x06fa3dd6e12c9a66aeacd2eefa5a2ff2915dd1bb4207596de29bd0e8cdeeae66_5)
- [Liquidation](https://starkscan.co/event/0x07b8ec709df1066d9334d56b426c45440ca1f1bb841285a5d7b33f9d1008f256_5)

In [ ]:
# TODO: convert numbers/amounts (divide by sth)
# TODO: remove irrelevant variables (in events)
# TODO: add self.user to UserState and UserTokenState?
# TODO: add logs
class AccumulatorState:
    '''
    TODO
    '''

    def __init__(self) -> None:
        self.lending_accumulator: decimal.Decimal = decimal.Decimal('1e27')
        self.debt_accumulator: decimal.Decimal = decimal.Decimal('1e27')

    def accumulators_sync(self, lending_accumulator: decimal.Decimal, debt_accumulator: decimal.Decimal):
        self.lending_accumulator = lending_accumulator / decimal.Decimal('1e27')
        self.debt_accumulator = debt_accumulator / decimal.Decimal('1e27')


class UserTokenState:
    '''
    TODO

    We are making a simplifying assumption that when collateral is enabled, all
    deposits of the given token are considered as collateral.
    '''

    # TODO: make it token-dependent (advanced solution: fetch token prices in $ -> round each token's 
    #   balance e.g. to the nearest cent)
    MAX_ROUNDING_ERROR = decimal.Decimal('1')

    def __init__(self, token: str) -> None:
        self.token: str = token
        self.deposit: decimal.Decimal = decimal.Decimal('0')
        self.collateral_enabled: bool = False
        self.borrowings: decimal.Decimal = decimal.Decimal('0')

    def update_deposit(self, raw_amount: decimal.Decimal):
        self.deposit += raw_amount
        if -self.MAX_ROUNDING_ERROR < self.deposit < self.MAX_ROUNDING_ERROR:
            self.deposit = decimal.Decimal('0')


class UserState:
    '''
    TODO
    '''

    def __init__(self) -> None:
        self.token_states: Dict[str, UserTokenState] = {
            'ETH': UserTokenState('ETH'),
            'wBTC': UserTokenState('wBTC'),
            'USDC': UserTokenState('USDC'),
            'DAI': UserTokenState('DAI'),
            'USDT': UserTokenState('USDT'),
        }
        # TODO: implement healt_factor
        # TODO: use decimal
        self.health_factor: float = 1.0  # TODO: is this a good default value??

    def deposit(self, token: str, raw_amount: decimal.Decimal):
        self.token_states[token].update_deposit(raw_amount)

    def withdrawal(self, token: str, raw_amount: decimal.Decimal):
        self.token_states[token].update_deposit(-raw_amount)

    def collateral_enabled(self, token: str):
        self.token_states[token].collateral_enabled = True

    def collateral_disabled(self, token: str):
        self.token_states[token].collateral_enabled = False

    def borrowing(self, token: str, raw_amount: decimal.Decimal, face_amount: decimal.Decimal):
        self.token_states[token].borrowings += raw_amount

    def repayment(self, token: str, raw_amount: decimal.Decimal, face_amount: decimal.Decimal):
        self.token_states[token].borrowings -= raw_amount

    def liquidation(
        self,
        debt_token: str,
        debt_raw_amount: decimal.Decimal,
        debt_face_amount: decimal.Decimal,
        collateral_token: decimal.Decimal,
        collateral_raw_amount: decimal.Decimal,
    ):
        self.token_states[debt_token].borrowings -= debt_raw_amount
        self.token_states[collateral_token].update_deposit(-collateral_raw_amount)


class State:
    '''
    TODO
    '''

    EVENTS_FUNCTIONS_MAPPING: Dict[str, str] = {
        'Deposit': 'process_deposit_event',
        'Withdrawal': 'process_withdrawal_event',
        'CollateralEnabled': 'process_collateral_enabled_event',
        'CollateralDisabled': 'process_collateral_disabled_event',
        'Borrowing': 'process_borrowing_event',
        'Repayment': 'process_repayment_event',
        'Liquidation': 'process_liquidation_event',
        'AccumulatorsSync': 'process_accumulators_sync_event',
    }

    def __init__(self) -> None:
        self.user_states: collections.defaultdict = collections.defaultdict(UserState)
        self.accumulator_states: Dict[str, AccumulatorState] = {
            'ETH': AccumulatorState(),
            'wBTC': AccumulatorState(),
            'USDC': AccumulatorState(),
            'DAI': AccumulatorState(),
            'USDT': AccumulatorState(),
        }

    def process_event(self, event: pandas.Series) -> None:
        name = event['key_name']
        getattr(self, self.EVENTS_FUNCTIONS_MAPPING[name])(event = event)

    def process_deposit_event(self, event: pandas.Series) -> None:
        # The order of the arguments is: `user`, `token`, `face_amount`.
        user = event['data'][0]
        token = constants.get_symbol(event['data'][1])
        # TODO: divide by something or store like this?
        # TODO: any better conversion to decimals?
        face_amount = decimal.Decimal(str(int(event['data'][2], base = 16)))
        # TODO: sanity checks/asserts?
        raw_amount = face_amount / self.accumulator_states[token].lending_accumulator
        self.user_states[user].deposit(token = token, raw_amount = raw_amount)

    def process_withdrawal_event(self, event: pandas.Series) -> None:
        # The order of the arguments is: `user`, `token`, `face_amount`.
        user = event['data'][0]
        token = constants.get_symbol(event['data'][1])
        face_amount = decimal.Decimal(str(int(event['data'][2], base = 16)))
        raw_amount = face_amount / self.accumulator_states[token].lending_accumulator
        self.user_states[user].withdrawal(token = token, raw_amount = raw_amount)

    def process_collateral_enabled_event(self, event: pandas.Series) -> None:
        # The order of the arguments is: `user`, `token`.
        user = event['data'][0]
        token = constants.get_symbol(event['data'][1])
        self.user_states[user].collateral_enabled(token = token)

    def process_collateral_disabled_event(self, event: pandas.Series) -> None:
        # The order of the arguments is: `user`, `token`.
        user = event['data'][0]
        token = constants.get_symbol(event['data'][1])
        self.user_states[user].collateral_disabled(token = token)

    def process_borrowing_event(self, event: pandas.Series) -> None:
        # The order of the arguments is: `user`, `token`, `raw_amount`, `face_amount`.
        user = event['data'][0]
        token = constants.get_symbol(event['data'][1])
        raw_amount = decimal.Decimal(str(int(event['data'][2], base = 16)))
        face_amount = decimal.Decimal(str(int(event['data'][3], base = 16)))  # TODO: relevant?
        self.user_states[user].borrowing(
            token = token,
            raw_amount = raw_amount,
            face_amount = face_amount,
        )

    def process_repayment_event(self, event: pandas.Series) -> None:
        # The order of the arguments is: `repayer`, `beneficiary`, `token`, `raw_amount`,
        # `face_amount`.
        repayer = event['data'][0]  # TODO: relevant?
        beneficiary = event['data'][1]
        token = constants.get_symbol(event['data'][2])
        raw_amount = decimal.Decimal(str(int(event['data'][3], base = 16)))
        face_amount = decimal.Decimal(str(int(event['data'][4], base = 16)))  # TODO: relevant?
        self.user_states[beneficiary].repayment(
            token = token,
            raw_amount = raw_amount,
            face_amount = face_amount,
        )

    def process_liquidation_event(self, event: pandas.Series) -> None:
        # The order of the arguments is: `liquidator`, `user`, `debt_token`, `debt_raw_amount`,
        # `debt_face_amount`, `collateral_token`, `collateral_amount`.
        liquidator = event['data'][0]  # TODO: relevant?
        user = event['data'][1]
        debt_token = constants.get_symbol(event['data'][2])
        debt_raw_amount = decimal.Decimal(str(int(event['data'][3], base = 16)))
        debt_face_amount = decimal.Decimal(str(int(event['data'][4], base = 16)))  # TODO: relevant?
        collateral_token = constants.get_symbol(event['data'][5])
        collateral_amount = decimal.Decimal(str(int(event['data'][6], base = 16)))
        collateral_raw_amount = collateral_amount / self.accumulator_states[collateral_token].lending_accumulator
        self.user_states[user].liquidation(
            debt_token = debt_token,
            debt_raw_amount = debt_raw_amount,
            debt_face_amount = debt_face_amount,
            collateral_token = collateral_token,
            collateral_raw_amount = collateral_raw_amount,
        )

    def process_accumulators_sync_event(self, event: pandas.Series) -> None:
        # The order of the arguments is: `token`, `lending_accumulator`, `debt_accumulator`.
        token = constants.get_symbol(event['data'][0])
        lending_accumulator = decimal.Decimal(str(int(event['data'][1], base = 16)))
        debt_accumulator = decimal.Decimal(str(int(event['data'][2], base = 16)))
        self.accumulator_states[token].accumulators_sync(
            lending_accumulator = lending_accumulator,
            debt_accumulator = debt_accumulator,
        )

In [ ]:
# Iterate over ordered events to obtain the final state of each user.
state = State()
zklend_events.sort_values(['block_number', 'event_index'], inplace = True)
for _, event in zklend_events.iterrows():
    state.process_event(event = event)

# TODO: Debugging negative deposits below

In [ ]:
# TODO: negative deposits?
for attr in ['deposit', 'borrowings']:
    for token in ['ETH', 'wBTC', 'USDC', 'DAI', 'USDT']:
        values = [getattr(user_state.token_states[token], attr) for user, user_state in state.user_states.items()]
        print(attr, token, pandas.Series(values).min())

In [ ]:
# TODO: show negative deposits
[
    (user, user_state.token_states['USDT'].deposit)
    for user, user_state
    in state.user_states.items()
    if user_state.token_states['USDT'].deposit < 0
]

In [ ]:
# TODO: Get token prices here: https://www.coingecko.com/en/api/documentation